In [1]:
def read_input(): 
    with open(f"input.txt", "r") as f:
        return f.read().split("\n")

In [2]:
from pathlib import PurePosixPath
from itertools import chain

# Data Structure

class MyFile:
    is_dir = False

    def __init__(self, name, size):
        self.name = name
        self.size = int(size)
    def get_size(self):
        return self.size
    
    def __repr__(self):
        return f"file: {self.name} {self.size}"
    
class Directory:
    is_dir = True
    def __init__(self, name):
        self.name = name
        self.children = {}
    
    def add_dir(self, dir_name):
        self.children[dir_name] = Directory(dir_name)
    
    def add_file(self, file_name, file_size):
        self.children[file_name] = MyFile(file_name, file_size)
        
    def get_size(self):
        return sum(x.get_size() for x in self.children.values())
    
    def get_dir(self, dir_name):
        return self.children[dir_name]
    
    def list_child_directories(self):
        direct_children = [x for x in self.children.values() if x.is_dir] 
        return direct_children + list(chain.from_iterable(x.list_child_directories() for x in direct_children))
        
    def __repr__(self):
        return f"directory: {self.name}"
    
## File System Operations

def get_directory_from_path(systemRoot, path):
    current_dir = systemRoot
    for part in path.parts[1:]:
        current_dir = current_dir.get_dir(part)
    return current_dir
        
def change_path(current_path, path):
    if path == "/":
        return PurePosixPath("/")
    elif path == "..":
        return current_path.parent
    else:
        return current_path / path
    
def add_dir(systemRoot, cwd, directory_name):
    current_dir = get_directory_from_path(systemRoot, cwd)
    current_dir.add_dir(directory_name)
    
def add_file(systemRoot, cwd, file_name, file_size):
    current_dir = get_directory_from_path(systemRoot, cwd)
    current_dir.add_file(file_name, file_size)

    
## Parse input lines

def parse_command(line, current_path):
    # cd, ls
    parts = line.split(" ")
    if parts[1] == "cd":
        directory_name = parts[2]
        return change_path(current_path, directory_name)
    return current_path
    

def parse_output(line, root, current_dir):
    parts = line.split(" ")
    if parts[0] == "dir":
        directory_name = parts[1] 
        add_dir(root, current_dir, directory_name)
    else:
        file_size, file_name = parts
        add_file(root, current_dir, file_name, file_size)

# Builder

def build_file_system():
    root = Directory("/")
    cwd = PurePosixPath("/")
    for line in read_input():
        if line.startswith("$"):
            cwd = parse_command(line, cwd)
        else:
            parse_output(line, root, cwd)
    return root
    

In [3]:
# Part 1

root = build_file_system()
all_directories = root.list_child_directories()
sum(x.get_size() for x in all_directories if x.get_size() <= 100000)

1783610

In [4]:
# Part 2

TOTAL_SPACE = 70000000 
REQUIRED_SPACE = 30000000
used_space = root.get_size()
free_space = TOTAL_SPACE - used_space
size_to_delete = REQUIRED_SPACE - free_space
sorted(x.get_size() for x in all_directories if x.get_size() >= size_to_delete)[0]

4370655